In [73]:
# Code Source article : https://www.analyticsvidhya.com/blog/2019/03/learn-to-use-elmo-to-extract-features-from-text/

In [74]:
import sys
!{sys.executable} -m pip install spacy

In [75]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
import re
import time
import pickle
#pd.set_option('display.max_colwidth', 200)

In [76]:
data = pd.read_csv("./data.csv")
#data = pd.read_csv("/home/saksham/Desktop/BTP/data_collecion/data_cleaned_segregated.csv")
data.columns = ["URL", "Text", "Start-Indices", "End-Indices", "Text-Type"]

In [77]:
from sklearn.model_selection import train_test_split
X = data["Text"]
y=data["Text-Type"]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1, random_state=1)
print("Training set size:", X_train.shape)
print("Test set size:", X_test.shape)

Training set size: (21255,)
Test set size: (2362,)


In [78]:
import sys
!{sys.executable} -m pip install "tensorflow>=1.7.0"
import sys
!{sys.executable} -m pip install tensorflow-hub

In [79]:
import tensorflow_hub as hub
import tensorflow as tf
# Clear the TensorFlow graph
tf.compat.v1.reset_default_graph()
tf.compat.v1.disable_eager_execution()
elmo = hub.load("https://tfhub.dev/google/elmo/3")

In [80]:
def elmo_vectors(x):
    embeddings =  embeddings = elmo.signatures["default"](tf.constant(x))["elmo"]
    return embeddings
#   with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     sess.run(tf.tables_initializer())
#     # return average of ELMo features
#     return sess.run(tf.reduce_mean(embeddings,1))

In [81]:
list_train = [X_train[i:i+100] for i in range(0,X_train.shape[0],100)]
list_test = [X_test[i:i+100] for i in range(0,X_test.shape[0],100)]

In [82]:
elmo_train = [elmo_vectors(x) for x in list_train]
elmo_test = [elmo_vectors(x) for x in list_test]

In [83]:
# elmo_train_new = np.concatenate(elmo_train, axis = 0)
# elmo_test_new = np.concatenate(elmo_test, axis = 0)

# # save elmo_train_new
# pickle_out = open("elmo_train_03032019.pickle","wb")
# pickle.dump(elmo_train_new, pickle_out)
# pickle_out.close()

# # save elmo_test_new
# pickle_out = open("elmo_test_03032019.pickle","wb")
# pickle.dump(elmo_test_new, pickle_out)
# pickle_out.close()

In [84]:

from sklearn.model_selection import train_test_split

xtrain, xvalid, ytrain, yvalid = train_test_split(elmo_train_new, 
                                                  train['label'],  
                                                  random_state=42, 
                                                  test_size=0.2)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

lreg = LogisticRegression()
lreg.fit(xtrain, ytrain)

NameError: name 'elmo_train_new' is not defined

In [ ]:
# make predictions on test set
preds_test = lreg.predict(elmo_test_new)

In [ ]:

# prepare submission dataframe
sub = pd.DataFrame({'id':test['id'], 'label':preds_test})

# write predictions to a CSV file
sub.to_csv("sub_lreg.csv", index=False)